In [2]:
import sys 
import warnings
import itertools 
import pandas as pd 
import altair as alt 
import numpy as np 
from typing import List, Union, Optional, Dict, Callable 
from collections import defaultdict 
from altair import datum 
from IPython.display import display, HTML
from scipy.signal import argrelextrema
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.utils import check_random_state
from sklearn.linear_model import Ridge, ElasticNet, LinearRegression
from sklearn.model_selection import cross_validate, ShuffleSplit, StratifiedKFold, StratifiedShuffleSplit, cross_val_score, GridSearchCV 
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline

warnings.filterwarnings("ignore")
# warnings.filterwarnings("always")

In [4]:
def layer_charts(*plots): 
    """ Overlay all charts in the input into a single output chart 
    """
    assert len(plots) >= 1
    c = plots[0]
    for p in plots[1:]: 
        c = c + p 
    return c 

def create_scatter(x, y):
    xmax = np.max(x)
    df = pd.DataFrame(data=dict(place_in_line=np.ravel(x), price_per_pod=np.ravel(y)))
    return alt.Chart(df, width=800).mark_point().encode(
        x=alt.X(
            "place_in_line:Q"
            , scale=alt.Scale(domain=(0, xmax))
        )
        , y=alt.Y(
            "price_per_pod:Q"
            , scale=alt.Scale(domain=(0, 1), clamp=True)
        )
#         , color=alt.Color(
#             'amount:Q'
#             , scale=alt.Scale(
#                 domain=(0, 150000)
#                 , range=('#71f0df', '#0023bd')
#                 , clamp=True
#             )
#         )
    )

def create_lines(df): 
    xmax = np.max(df.place_in_line.values)
    selection = alt.selection_multi(fields=['hyp_params'], bind='legend')
    return alt.Chart(df, width=800).mark_line().encode(
        x=alt.X(
            "place_in_line:Q"
            , scale=alt.Scale(domain=(0, xmax))
        )
        , y=alt.Y(
            "price_per_pod:Q"
            , scale=alt.Scale(domain=(0, 1), clamp=True)
        )
        , color=alt.Color(
            "hyp_params:N"
            , legend=alt.Legend(title="Hyper Parameters", orient="bottom", direction="vertical")
        )
        , opacity=alt.condition(selection, alt.value(1), alt.value(0.025))
    ).add_selection(
        selection
    )

def get_chart_scores(df_scores): 
    dmin = np.min(
        np.union1d(df_scores.train_nmse.values, df_scores.test_nmse)
    )
    dmax = np.max(
        np.union1d(df_scores.train_nmse, df_scores.test_nmse)
    )
    yscale = alt.Scale(domain=(dmin, dmax))
    scores = alt.Chart(df_scores, title="Training vs Testing Negative-MSE (higher is better)").mark_line().transform_fold(
        fold=['test_nmse', 'train_nmse'], 
        as_=['type', 'nmse']
    ).encode(
        x="alpha:Q",
        y=alt.Y("nmse:Q", scale=yscale), 
        color='type:N'
    )
    return scores 

def get_chart_weights_boxplot(df_weights): 
    wlr_df = pd.melt(
        df_weights
        , value_vars=df_weights.columns
        , var_name="feature"
        , value_name="weight"
    )
    return alt.Chart(wlr_df).mark_boxplot(extent='min-max').encode(
        x='weight:Q'
        , y='feature:O'
    )

def ddf(df): 
    display(HTML(df.to_html()))

In [5]:
def get_df(nbins=4): 
    """ Get the underlying dataframe for the regression problem 
    
    Args: 
        nbins: The number of bins to be used for stratification 
    """
    file = './pod-market-history.csv'
    df = pd.read_csv(file)
    df['stratify'] = pd.cut(df['place_in_line'], nbins, labels=False)
    return df

def get_data(
    df 
    , features: Union[List[str], Dict[str, str]]
    , target_feature: str
    , st_feature: str
    , sw_feature: str 
    , sw_preprocess: Optional[Callable] = None 
): 
    """ Extract data from dataframe to be used for regression problem 
    
    Args: 
        df: the dataframe from which we extract our data
        features: 
            if features is a list, we use these columns as our feature set, with the corresponding names 
            if features is a dict, we extract columns using the keys and name the features using the corresponding value, enabling aliasing
        target_feature: the feature to use as the target variable. 
        st_feature: the feature to use for stratified k fold cross validation 
        sw_feature: the feature to use as our sample weight 
        sample_weight_preprocess: optional function to apply a transformation to the column we are using for sample weights 
    Returns: 
        X: shape is (len(df), len(features))
        y: shape is (len(df),)
        sw: shape is (len(df),)
        stratify: shape is (len(df),)
    """
    if isinstance(features, list): 
        feature_names = features
        X = df.loc[:,features].values
    elif isinstance(features, dict): 
        feature_names = list(features.values()) 
        X = df.loc[:,list(features.keys())].values 
    else: 
        raise Exception("`features` must either be a list or a dict")
    y = df[target_feature].values 
    stratify = df[st_feature].values  
    sw = df[sw_feature].values 
    if sw_preprocess: 
        sw = sw_preprocess(sw)
    return X, y, sw, stratify, feature_names

In [51]:
def fit_predict(model, X, y, sample_weights, train_idx=None, test_idx=None): 
    if train_idx is not None and test_idx is not None: 
        X_train, y_train, w_train = X[train_idx], y[train_idx], sample_weights[train_idx]
        X_test, y_test, w_test = X[test_idx], y[test_idx], sample_weights[test_idx]
    else: 
        X_train, y_train, w_train = X, y, sample_weights 
        X_test, y_test, w_test = X, y, sample_weights 
    y_train_pred = model.fit(X_train, y_train, regressor__sample_weight=w_train) 
    y_test_pred = model.fit(X_test, y_test, regressor__sample_weight=w_test) 
    train_score = mean_squared_error(y_train, model.predict(X_train), sample_weight=w_train)
    test_score = mean_squared_error(y_test, model.predict(X_test), sample_weight=w_test)
    weights = model[-1].coef_
    return y_train_pred, y_test_pred, weights, train_score, test_score 

class CustomStratifiedKFold(): 
    
    def __init__(self, n_splits, random_state, stratify): 
        self.stratify = stratify
        self.n_splits = n_splits 
        self.cv = StratifiedKFold(
            n_splits=self.n_splits, 
            random_state=random_state, 
            shuffle=True 
        )
        
    def get_folds(self, X): 
        assert X.shape[0] == self.stratify.shape[0] 
        for train_idx, test_idx in self.cv.split(X, self.stratify):
            yield train_idx, test_idx 
        
def run_experiment(
    X
    , y
    , stratify
    , sw 
    , feature_names: List[str] 
    , degrees: List[int]
    , alphas: List[float]
    , model_params 
    , train_p: float
    , row_height: int = 300 
    , half_width: int = 600 
    , n_fit_samples: int = 50
): 
    """
    args: 
        degrees: values of degree to test for hyperparameter optimality 
        alphas: values of alpha to test for hyperparameter optimality 
        train_percent: values of train_percent to test for hyperparameter optimality 
    returns: 
        alt.Chart object containg all plots to be used for model validation 
    """
    rs = 42
    scoring = 'neg_mean_squared_error'
    rows = []
    dfs = defaultdict(list) 
    # Train set is used for cross validation of hyperparameters, test set is used to evaluate final model performance 
    idx_train, idx_test = next(
        StratifiedShuffleSplit(
            n_splits=1
            , random_state=rs
            , train_size=train_p
        ).split(X, stratify)
    )
    X_train, X_test = X[idx_train], X[idx_test] 
    y_train, y_test = y[idx_train], y[idx_test]
    stratify_train, stratify_test = stratify[idx_train], stratify[idx_test] 
    sw_train, sw_test = sw[idx_train], sw[idx_test] 
    model = Pipeline([
        ('polyfeatures', PolynomialFeatures()), 
        ('scaler', StandardScaler()), # since polynomial features have wildly different ranges, scaling is important 
        ('regressor', Ridge(**model_params)) 
    ])
    for degree in degrees: 
        alpha_train_scores = dict()
        alpha_test_scores = dict()
        alpha_weights = dict()
        for i, alpha in enumerate(alphas): 
            nfolds = 4
            cv = CustomStratifiedKFold(nfolds, rs, stratify_train).get_folds(X_train)
            model = Pipeline([
                ('polyfeatures', PolynomialFeatures(degree=degree)), 
                ('scaler', StandardScaler()), # since polynomial features have wildly different ranges, scaling is important 
                ('regressor', Ridge(**model_params, alpha=alpha)) 
            ])        
            scores = cross_validate(
                model
                , X_train
                , y_train
                , cv=cv
                , scoring=scoring
                , fit_params=dict(regressor__sample_weight=sw_train)
                , return_train_score=True 
                , return_estimator=True 
            )
            alpha_train_scores[alpha] = scores['train_score']
            alpha_test_scores[alpha] = scores['test_score']
            alpha_weights[alpha] = np.mean([est[-1].coef_ for est in scores['estimator']], axis=0)
  
            # set the model weights to the average weights across all folds 
            # retrain this model on the whole training set so we can compare generalization performance 
            model.fit(X_train, y_train, regressor__sample_weight=sw_train) 
            
            Xfit = np.linspace(0, np.max(X[:,0]), num=n_fit_samples).reshape((n_fit_samples, X.shape[1]))
            yfit = model.predict(Xfit)
            c = f'0{i}' if i < 10 else str(i)
            pstr = f'{c}. Degree: {degree} Alpha: {alpha:.15f}'
            dfs[degree].append(
                pd.DataFrame(
                    data=dict(
                        place_in_line=np.ravel(Xfit)
                        , price_per_pod=np.ravel(yfit)
                        , hyp_params=np.array([pstr for i in range(len(Xfit))])
                    )
                )
            )
            
        # construct dataframe containing training and testing scores, plus indicators of local minima status 
        asm_train = [np.mean(alpha_train_scores[alpha]) for alpha in alphas]
        asm_test = [np.mean(alpha_test_scores[alpha]) for alpha in alphas]
        lm_train_ind = argrelextrema(np.array(asm_train), np.less_equal, order=1)[0]
        lm_test_ind = argrelextrema(np.array(asm_test), np.less_equal, order=1)[0]
        lm_train = [True if i in lm_train_ind else False for i in range(len(asm_train))]
        lm_test = [True if i in lm_test_ind else False for i in range(len(asm_test))]
        df_scores = pd.DataFrame(
            data=dict(
                alpha=alphas, 
                train_nmse=asm_train, 
                test_nmse=asm_test, 
                lm_train=lm_train, 
                lm_test=lm_test
            ) 
        ) 
        
        chart_scores = get_chart_scores(df_scores).properties(
            width=half_width
            , height=row_height
        ).interactive() 
        
        # construct dataframe containing alphas vs model coefficients, visualize as a heatmap. 
        weight_rows = []
        for a, ws in alpha_weights.items(): 
            for w, f in zip(ws, scores['estimator'][0][0].get_feature_names_out(input_features=['place_in_line'])): 
                weight_rows.append([a, w, str(f)])
        df_weights = pd.DataFrame(data=weight_rows, columns=['alpha', 'weight', 'feature'])
        
        chart_weights = alt.Chart(df_weights).mark_rect().encode(
            x="alpha:O", 
            y="feature:O", 
            color=alt.Color("weight:Q", scale=alt.Scale(scheme='magma'))
        ).properties(
            width=half_width
            , height=row_height
        ).interactive() 
        
        rows.append(alt.hconcat(chart_scores, chart_weights)) 
        
        # best_alpha = df_alphas_mse_test.sort_values('alpha')[np.invert(np.isnan(df_alphas_mse_test.local_minima).values)]['alpha'].values[0]
    
    for d, ddfs in dfs.items(): 
        df = pd.concat(ddfs)
        chart_regression = layer_charts(*[
            create_scatter(X, y), 
            create_lines(df)
        ])
        rows.append(
            alt.hconcat(chart_regression.interactive()).resolve_legend(
                color="independent"
            ).resolve_scale(
                color="independent"
            )
        )
    
    return alt.vconcat(*rows).configure_view(continuousWidth=half_width*2)      

In [54]:
df = get_df()
features = ['place_in_line']
target_feature = 'price_per_pod'
st_feature = 'stratify'
sw_feature = 'amount'
# sw_preprocess = np.log10
sw_preporcess = None 
X, y, sw, stratify, feature_names = get_data(df, features, target_feature, st_feature, sw_feature, sw_preprocess) 

# visualize number of samples in each stratification cut 
# alt.Chart(df, width=600).mark_bar().encode(
#     x=alt.X('stratify:O', bin=True), 
#     y="count()"
# ).interactive()

degrees = [7, 8, 9, 10, 11]
alphas = np.logspace(-14, -3, num=30) # for ridge 
# alphas = np.linspace(1e-12, 1e-4, num=25) # for ridge 
train_percent = .98

# Degree 10 #9 
# Degree 11 #11 
run_experiment(X, y, stratify, sw, feature_names, degrees, alphas, dict(tol=1e-6), train_percent)

alt.VConcatChart(...)

In [8]:
# degree = 10
# alpha = 10e-11

# model = Pipeline([
#     ('polyfeatures', PolynomialFeatures(degree=degree)), 
#     ('scaler', StandardScaler()), # since polynomial features have wildly different ranges, scaling is important 
#     ('ridge', Ridge(alpha=alpha)) 
# ])
# y_train_pred, y_test_pred, weights, train_score, test_score = fit_predict(model, np.reshape(x, (len(x), 1)), y, sample_weights)

# nsamples = 50
# xfit = np.reshape(np.linspace(0, np.max(x), num=nsamples), (nsamples, 1))
# yfit = model.predict(xfit)

# xmax = np.max(x)
# df = pd.DataFrame(
#     data=dict(
#         place_in_line=np.ravel(xfit)
#         , price_per_pod=np.ravel(yfit)
#     )
# )

# layer_charts(*[
#     create_scatter(x, y), 
#     alt.Chart(df, width=800).mark_line().encode(
#         x=alt.X(
#             "place_in_line:Q"
#             , scale=alt.Scale(domain=(0, xmax))
#         )
#         , y=alt.Y(
#             "price_per_pod:Q"
#             , scale=alt.Scale(domain=(0, 1), clamp=True)
#         )
#     )
# ]).interactive() 

In [15]:
from itertools import product 


for i, (split_idx, (train, test)) in product(
    range(0, 5), enumerate(CustomStratifiedKFold(3, 1, stratify).get_folds(X))
): 
    # print(i, split_idx)
    if split_idx == 2: 
        print(train)

[  0   1   2   4   6   7   9  12  13  14  17  18  19  20  21  22  23  24
  25  26  27  30  31  32  34  35  37  38  39  40  42  43  44  45  48  49
  50  51  54  56  57  58  59  60  62  64  65  66  68  69  70  72  73  74
  75  77  78  81  83  85  86  88  89  93  94  95  96  98  99 100 101 102
 103 107 108 109 110 111 112 113 117 118 120 122 123 124 125 126 127 129
 131 134 135 136 137 138 139 140 143 144 145 146 147 148 149 150 151 154
 155 156 157 158 160 161 162 164 166 167 168 170 172 174 175 176 177 179
 180 183 185 186 188 190 192 193 197 200 201 202 206 207 208 209 212 214
 215 218 219 220 223 224 225 226 227 228 229 230 231 232 234 235 236 238
 240 241 242 245 248 249 250 252 254 255 256 258 259 260 262 264 265 267
 268 270 271 273 274 276 278 279 281 282 284 285 286 287 289 290 291 292
 293 297 299 301 302 304 305]
[  0   1   2   4   6   7   9  12  13  14  17  18  19  20  21  22  23  24
  25  26  27  30  31  32  34  35  37  38  39  40  42  43  44  45  48  49
  50  51  54  56  57 